# Import Libraries

In [2]:
################ WEB SCRAPING MODULES ############
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import bs4
from fake_useragent import UserAgent
import requests
################ TIME MODLULES ###################
import time
from datetime import date 
import datetime
############## DATA MANIPULATION MODULES #########
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords

# Read 100 questions list

In [3]:
df = pd.read_excel('AnswersFromSource11_20.xlsx',engine='openpyxl')
link = 'https://www.healthline.com/health/our-two-cents-doctors-answer-questions-about-autism'
df[link]=np.nan
#df.drop(df.iloc[:, 1:25], inplace = True, axis = 1)
df

,Question,https://springbrookbehavioral.com/frequently-asked-questions-about-autism/,https://www.aacap.org/AACAP/Families_and_Youth/Resource_Centers/Autism_Resource_Center/FAQ.aspx,https://www.autism.org.sg/living-with-autism/myths-and-facts-of-autism,https://www.autismsa.org.au/frequently-asked-questions-about-asd,https://www.bbrfoundation.org/faq/frequently-asked-questions-about-autism,https://www.cdc.gov/ncbddd/autism/topics.html,https://www.emedicinehealth.com/autism_spectrum_disorder_quiz_iq/faq.htm,https://www.healthline.com/health/our-two-cents-doctors-answer-questions-about-autism
0,What Can I Do To Prepare My Child For The Future?,Early intervention can do a lot toward fosteri...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Did I Cause My Child’s Autism?,"No. After getting an ASD diagnosis, this is th...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Can My Child’s Autism Be Cured?,No. Autism is not a disease that can be cured—...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,How Can I Deal With This Diagnosis?,"Start by being patient and kind with yourself,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,How Do I Find Help For My Child?,Start by making sure that you and your child h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
138,What are some ways that parents can reduce the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,Do some families deal with stress better than ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,Do siblings suffer increased stress as a resul...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,What can I do about my children’s stress?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Scrape QA pairs from website

In [4]:
# Open webpage in a new window
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.healthline.com/health/our-two-cents-doctors-answer-questions-about-autism')



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [/home/gopan/.wdm/drivers/chromedriver/linux64/92.0.4515.107/chromedriver] found in cache


In [5]:
# Parse text in webpage
source = driver.page_source
soup = bs4.BeautifulSoup(source, 'html.parser')

In [11]:
# Search the questions mentioned in webpage
l = []
for p in soup.find_all('p',{'class':'Body'}):
    for strong in p.find_all('strong'):
        if(strong.text[-1]=='?'):
            print(strong.text)
            l.append(strong.text)

How is autism diagnosed in young children?
What are early signs of autism?
Why do people diagnosed with autism spectrum disorder (ASD) sometimes have a difficult time making eye contact?
How does visual processing impact someone with ASD?
How can siblings be included in the care of children with autism and related disabilities?
What can I do because my child never listens and I seem to be always nagging?


In [12]:
# Tried to search for answers of questions
k=[]
c = 0
for p in soup.find_all('p',{'class':'Body'}):
    if p.find('strong'):
        continue
    txt = p.text.strip()
    if(txt in l):
        print('*'*100)
    if(txt in l or len(txt.split())>1):
        print(c,'-->', txt)
    c = c+1
    
    k.append(txt)

0 --> It’s estimated that 1.5 million people in the United States have autism spectrum disorder (ASD), while a recent CDC reportTrusted Source indicates a rise in autism rates. It’s more imperative than ever to increase our understanding and awareness of this disorder.
1 --> One way to do this is to understand the obstacles autism presents — not just for those who receive a diagnosis, but for the entire family. We looked to three doctors who shared and answered some of the more common questions they get asked about autism.
2 --> From how a child is diagnosed, to how autism can affect the family dynamic, read on to find out what they had to say.
3 --> The diagnosis of autism is based on an expert clinician’s careful observations of a child’s behavior. The clinician engages the child in a set of play activities that are designed to probe for autism symptoms, and the diagnosis is based on how many symptoms are present.
4 --> A certain number of symptoms in two categories are required: dif

In [16]:
# Tried to search for answers of questions
li=[]
c = 0
for ul in soup.find_all('li'):
    txt = ul.text.strip()
    if(txt in l):
        print('*'*100)
    if(txt in l or len(txt.split())>1):
        print(c,'-->', txt)
    c = c+1
    
    li.append(txt)

0 --> Health Conditions
6 --> Health & Well-Being
8 --> Sex & Relationships
9 --> Products & Gear
12 --> reduced interest in people
13 --> lack of gestures such as pointing and showing
14 --> lack of engagement in social play, such as “patty cake”
15 --> failure to consistently orient when the child’s name is called
16 --> eye tics or blinking
17 --> dilated pupils
18 --> erratic eye movements
19 --> poor eye contact or avoiding eye contact
20 --> avoiding visual attention, especially reading and near work
21 --> frequent loss of place when reading
22 --> rereading letters or words
23 --> closing or blocking one eye when reading
24 --> looking out of the corner of the eye
25 --> difficulty copying from far away
26 --> holding a book too close to the eyes
27 --> overly interested in shadows, patterns, or lights
28 --> bumping or running into objects
29 --> confusion going up or down stairs
33 --> About AFAA: Autism on the rise. (n.d.).afaa-us.org/about
34 --> Baio J, et al. (2018). Prev

# Check which questions are similar

In [13]:
stop_words = set(stopwords.words('english'))

def clean_text(sent):
    sent = sent.lower() # lowercase
    sent = re.sub(r'[^\w\s]', '', sent) # remove punctuations
    sent = re.sub('Autism Spectrum Disorder','ASD',sent) # Compress term
    sent = [w for w in sent.split() if not w.lower() in stop_words] # Remove stopwords
    sent = " ".join(sent)
    return sent

In [14]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [15]:
for i in l:
    i2 = clean_text(i)
    list1 = i2.split()
    
    temp1 = 0.0
    temp2 = ''
    temp3 = ''
    count = 0
    for z1, j in enumerate(df['Question']):
        j2 = clean_text(j)
        list2 = j2.split()
            
        sim = jaccard_similarity(list1, list2)
        
        if(sim>temp1):
            temp1 = sim
            temp2 = j
            temp3 = i
            count = z1+1
    if(temp1>=0.3):
        print('Website --> ',temp3)
        print('100 questions[ ',count,' ] list --> ',temp2)
        print(temp1)
        print('*'*100)

Website -->  How is autism diagnosed in young children?
100 questions[  53  ] list -->  What is the prognosis of children with Autism?
0.4
****************************************************************************************************
Website -->  What are early signs of autism?
100 questions[  12  ] list -->  What are early signs of autism? 
1.0
****************************************************************************************************


In [20]:
# Directly add answer to 'What are the Autism Spectrum Disorders (ASD)?' in existing dataframe
df.iloc[11,0]
df[link].loc[11] = k[5] + '\n'+ li[12]+ '\n'+ li[13]+ '\n'+ li[14]+ '\n'+ li[15]+ '\n'+ k[6]

/home/gopan/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [24]:

l1 = [l[0],l[2],l[3],l[4],l[5]]
l1

['How is autism diagnosed in young children?',
 'Why do people diagnosed with autism spectrum disorder (ASD) sometimes have a difficult time making eye contact?',
 'How does visual processing impact someone with ASD?',
 'How can siblings be included in the care of children with autism and related disabilities?',
 'What can I do because my child never listens and I seem to be always nagging?']

In [44]:
#l=l[1:]
#l

In [25]:
# manually select answers to questions in list 'l'

#l2 = [k[8]+'\n'+k[12]+'\n'+k[13]+'\n'+k[14],
#      k[16]+'\n'+k[17],
#     k[19]+'\n'+k[21]]
l2 = [k[3]+'\n'+k[4],
      k[7]+'\n'+k[8],
      k[7]+'\n'+k[8]+'\n'+li[16]+'\n'+li[17]+'\n'+li[18]+'\n'+li[19]+'\n'+li[20]+'\n'+li[21]+'\n'+li[22]+'\n'+li[23]+'\n'+li[24]+'\n'+li[25]+'\n'+li[26]+'\n'+li[27],
      k[11]+'\n'+k[12]+'\n'+k[13],
     k[14]+'\n'+k[15]+'\n'+k[16]+'\n'+k[17]]

In [27]:
# Create new dataframe with QA pairs
df2 = pd.DataFrame(zip(l1,l2),columns=['Question',link])
df2

,Question,https://www.healthline.com/health/our-two-cents-doctors-answer-questions-about-autism
0,How is autism diagnosed in young children?,The diagnosis of autism is based on an expert ...
1,Why do people diagnosed with autism spectrum d...,Researchers recently have found that people di...
2,How does visual processing impact someone with...,Researchers recently have found that people di...
3,How can siblings be included in the care of ch...,Siblings of a child with a disability or illne...
4,What can I do because my child never listens a...,This is a very common concern for parents of c...


In [28]:
# Concatenate existing and new dataframes
df3 = pd.concat([df,df2],axis=0)
df3 = df3.sort_values(by=list(df3.columns[1:])).reset_index(drop=True)
df3

,Question,https://springbrookbehavioral.com/frequently-asked-questions-about-autism/,https://www.aacap.org/AACAP/Families_and_Youth/Resource_Centers/Autism_Resource_Center/FAQ.aspx,https://www.autism.org.sg/living-with-autism/myths-and-facts-of-autism,https://www.autismsa.org.au/frequently-asked-questions-about-asd,https://www.bbrfoundation.org/faq/frequently-asked-questions-about-autism,https://www.cdc.gov/ncbddd/autism/topics.html,https://www.emedicinehealth.com/autism_spectrum_disorder_quiz_iq/faq.htm,https://www.healthline.com/health/our-two-cents-doctors-answer-questions-about-autism
0,What Can I Do To Prepare My Child For The Future?,Early intervention can do a lot toward fosteri...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Did I Cause My Child’s Autism?,"No. After getting an ASD diagnosis, this is th...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Can My Child’s Autism Be Cured?,No. Autism is not a disease that can be cured—...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,How Can I Deal With This Diagnosis?,"Start by being patient and kind with yourself,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,How Do I Find Help For My Child?,Start by making sure that you and your child h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
143,What are some ways that parents can reduce the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144,Do some families deal with stress better than ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,Do siblings suffer increased stress as a resul...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,What can I do about my children’s stress?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Save Output

In [29]:
df3.to_excel('AnswersFromSource11_20.xlsx',index=False)